---
# Tehnici de Optimizare pentru Seturi Mari de Date

Când lucrăm cu seturi de date mici, performanța nu este o problemă. Însă, în lumea reală, seturile de date pot avea milioane sau chiar miliarde de rânduri. În astfel de cazuri, memoria RAM și timpul de procesare devin resurse critice. Acest capitol se concentrează pe tehnici esențiale pentru a lucra eficient cu date de mari dimensiuni în Pandas, asigurându-ne că codul nostru rulează rapid și fără a epuiza memoria calculatorului.

In [1]:
# Exemplu: Crearea unui DataFrame pentru exemplele noastre
import pandas as pd
import numpy as np

date_angajati = {
    'Nume': ['Andrei Popescu', 'Maria Ionescu', 'Ion Gheorghe', 'Elena Vasile', 'Vasile Costin'],
    'Departament': ['IT', 'Vânzări', 'IT', 'Marketing', 'Vânzări'],
    'Salariu_Brut_EUR': [3000, 2500, 2800, 2200, 2600],
    'Vechime_ani': [5, 3, 4, 2, 4]
}

df_angajati = pd.DataFrame(date_angajati)

print("DataFrame-ul de angajați:")
print(df_angajati)

print()

date_produse = {
    'Produs': ['Tricou', 'Pantaloni', 'Adidași', 'Șapcă', 'Geacă', 'Hanorac'],
    'Categorie': ['Îmbrăcăminte', 'Îmbrăcăminte', 'Încălțăminte', 'Accesorii', 'Îmbrăcăminte', 'Îmbrăcăminte'],
    'Pret_RON': [80, 250, 400, 50, 500, np.nan],
    'Stoc': [100, 50, 75, 200, 25, 40]
}

df_produse = pd.DataFrame(date_produse)

print("DataFrame-ul de produse:")
print(df_produse)
print()
print(df_produse.info())

DataFrame-ul de angajați:
             Nume Departament  Salariu_Brut_EUR  Vechime_ani
0  Andrei Popescu          IT              3000            5
1   Maria Ionescu     Vânzări              2500            3
2    Ion Gheorghe          IT              2800            4
3    Elena Vasile   Marketing              2200            2
4   Vasile Costin     Vânzări              2600            4

DataFrame-ul de produse:
      Produs     Categorie  Pret_RON  Stoc
0     Tricou  Îmbrăcăminte      80.0   100
1  Pantaloni  Îmbrăcăminte     250.0    50
2    Adidași  Încălțăminte     400.0    75
3      Șapcă     Accesorii      50.0   200
4      Geacă  Îmbrăcăminte     500.0    25
5    Hanorac  Îmbrăcăminte       NaN    40

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Produs     6 non-null      object 
 1   Categorie  6 non-null      object 
 2   Pret_RON   5 no

## Folosirea Tipurilor de Date Eficiente

Când Pandas încarcă date, alocă automat tipuri de date pentru fiecare coloană. De exemplu, pentru numere întregi folosește `int64`, iar pentru cele raționale `float64`. Aceste tipuri de date folosesc 64 de biți pentru a stoca fiecare valoare, ceea ce poate fi un consum inutil de memorie dacă valorile din coloană sunt mici.

Putem reduce semnificativ consumul de memorie prin conversia coloanelor la tipuri de date mai mici (ex: `int32`, `float32`) sau la tipuri specializate, cum ar fi **`category`**.

Tipul `category` este ideal pentru coloanele care au un număr redus de valori unice (de exemplu, 'Departament', 'Țară', 'Status'). În loc să stocheze textul de nenumărate ori, Pandas stochează doar valorile unice și apoi folosește numere întregi (coduri) pentru a face referire la ele, economisind astfel foarte mult spațiu.

In [3]:
# Exemplu 1: Verificarea consumului de memorie și optimizarea tipurilor de date

print("Consumul de memorie inițial:")
print(df_angajati.memory_usage(deep=True))

# Acum, să optimizăm tipurile de date.
df_optim = df_angajati.copy()

# Coloana 'Departament' este un candidat perfect pentru tipul 'category'.
df_optim['Departament'] = df_optim['Departament'].astype('category')

# 0 IT
# 1 IT
# 2 Marketing
# 3 Marketing
# 4 Marketing
# 5 Marketing

# -> IT Marketing
# -> IT > 0 | Marketing > 1

# 0 0
# 1 0
# 2 1
# 3 1
# 4 1
# 5 1


# Salariile pot fi stocate ca float32, iar vechimea ca int8 (un întreg pe 8
# biți, -128 la 127).
df_optim['Salariu_Brut_EUR'] = df_optim['Salariu_Brut_EUR'].astype('float32')
df_optim['Vechime_ani'] = df_optim['Vechime_ani'].astype('int8')

print("\nConsumul de memorie după optimizare:")
print(df_optim.memory_usage(deep=True))

# OBS.: `deep=True` asigură calcularea exactă a memoriei ocupate de obiectele
# de tip string, nu doar de referințele către ele.
# Reducerea memoriei poate fi dramatică pe seturi de date mari.

Consumul de memorie inițial:
Index               132
Nume                309
Departament         340
Salariu_Brut_EUR     40
Vechime_ani          40
dtype: int64

Consumul de memorie după optimizare:
Index               132
Nume                309
Departament         312
Salariu_Brut_EUR     20
Vechime_ani           5
dtype: int64


In [4]:
# __EXERCIȚIU__
# Creeați un DataFrame nou pe baza DataFrame-ului `df_produse`. Optimizați
# spațiul de stocare alocat setului de date schimbând tipul de date al unora
# dintre coloane.

print(df_produse.memory_usage(deep=True))
print()
print(df_produse.info())
print()

df_copie = df_produse.copy()

df_copie["Categorie"] = df_copie["Categorie"].astype("category")
df_copie["Pret_RON"] = df_copie["Pret_RON"].astype("float16")
df_copie["Stoc"] = df_copie["Stoc"].astype("int32")

print(df_copie.memory_usage(deep=True))

Index        132
Produs       401
Categorie    508
Pret_RON      48
Stoc          48
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Produs     6 non-null      object 
 1   Categorie  6 non-null      object 
 2   Pret_RON   5 non-null      float64
 3   Stoc       6 non-null      int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 324.0+ bytes
None

Index        132
Produs       401
Categorie    385
Pret_RON      12
Stoc          24
dtype: int64


## Procesarea în Bucăți (Chunk Processing)

Ce se întâmplă când un fișier este atât de mare încât nu încape deloc în memoria RAM? Răspunsul este procesarea în bucăți (**chunks**). În loc să citim tot fișierul deodată, îl citim pe bucăți de dimensiuni fixe (de exemplu, 10.000 de rânduri odată).

Acest lucru se realizează folosind parametrul `chunksize` din funcția `pd.read_csv()`. Aceasta nu mai returnează un DataFrame, ci un iterator, pe care putem să-l parcurgem cu o buclă `for`, procesând fiecare bucată individual. Această tehnică este esențială pentru a putea lucra cu seturi de date care depășesc capacitatea memoriei RAM.

In [5]:
# Exemplu: Simularea citirii unui fișier mare în bucăți

# Mai întâi, vom crea un fișier CSV demonstrativ.
df_angajati.to_csv('angajati.csv', index=False)

suma_salarii_totala = 0
numar_randuri_total = 0
dimensiune_bucata = 2  # Folosim o dimensiune mică pentru a demonstra conceptul

# Citim fișierul CSV în bucăți de câte 2 rânduri
for bucata in pd.read_csv('angajati.csv', chunksize=dimensiune_bucata):
    print("--- O nouă bucată a fost procesată ---")
    print(bucata)

    # Aplicăm o operație pe bucată: calculăm suma salariilor
    suma_salarii_totala += bucata['Salariu_Brut_EUR'].sum()
    numar_randuri_total += len(bucata)

salariu_mediu_calculat = suma_salarii_totala / numar_randuri_total

print(f"\nSalariul mediu calculat prin procesarea în bucăți: {salariu_mediu_calculat:.2f} EUR")

# OBS.: Această tehnică ne permite să calculăm statistici agregate (sumă, medie,
# etc.) pe un fișier întreg,
# fără a-l încărca complet în memorie. Păstrăm doar rezultatele intermediare
# (suma și numărul de rânduri).

--- O nouă bucată a fost procesată ---
             Nume Departament  Salariu_Brut_EUR  Vechime_ani
0  Andrei Popescu          IT              3000            5
1   Maria Ionescu     Vânzări              2500            3
--- O nouă bucată a fost procesată ---
           Nume Departament  Salariu_Brut_EUR  Vechime_ani
2  Ion Gheorghe          IT              2800            4
3  Elena Vasile   Marketing              2200            2
--- O nouă bucată a fost procesată ---
            Nume Departament  Salariu_Brut_EUR  Vechime_ani
4  Vasile Costin     Vânzări              2600            4

Salariul mediu calculat prin procesarea în bucăți: 2620.00 EUR


In [6]:
# __EXERCIȚIU__
# Folosind fișierul 'angajati.csv' și procesarea în bucăți, găsiți cea mai mare
# vechime în ani din întregul set de date.
# La final, afișați un mesaj: "Cea mai mare vechime este de [...] ani."

valoarea_maxima_globala = 0

dimensiune_bucata = 3

for bucata in pd.read_csv('angajati.csv', chunksize=dimensiune_bucata):
  valoarea_maxima_locala = bucata["Vechime_ani"].max()
  print(f"Valoarea maxima locala este: {valoarea_maxima_locala}")

  if valoarea_maxima_locala > valoarea_maxima_globala:
    valoarea_maxima_globala = valoarea_maxima_locala
    print(f"Noul maxim a devenit: {valoarea_maxima_globala}")

  print()

print(f"Cea mai mare vechime este de {valoarea_maxima_globala} ani.")

Valoarea maxima locala este: 5
Noul maxim a devenit: 5

Valoarea maxima locala este: 4

Cea mai mare vechime este de 5 ani.


---
# Capitolul: Lucrul cu Fișiere Mari și Formate Eficiente

Continuăm explorarea tehnicilor pentru date mari, concentrându-ne de data aceasta pe vizualizarea progresului operațiunilor de lungă durată și pe utilizarea unor formate de fișiere optimizate pentru viteză și spațiu. Când o operație durează minute sau chiar ore, este esențial să avem o idee despre stadiul ei. De asemenea, formatul în care salvăm datele poate avea un impact uriaș asupra performanței.

## Bare de Progres cu `tqdm`

Când procesăm un fișier mare în bucăți, poate dura mult timp, iar programul pare că a "înghețat". Pentru a oferi feedback vizual, putem folosi biblioteca **`tqdm`**. Aceasta se integrează foarte ușor cu buclele din Python și afișează o bară de progres inteligentă, care ne arată procentul de finalizare, timpul scurs și o estimare a timpului rămas.

! Dacă nu aveți `tqdm` instalat, rulați comanda `pip install tqdm` în terminal sau `!pip install tqdm` într-o celulă de cod.

In [11]:
# Exemplu: Adăugarea unei bare de progres la procesarea în bucăți

from tqdm import tqdm
import math

# Pentru ca tqdm să poată afișa progresul total, trebuie să știe numărul total
# de iterații. Vom calcula numărul total de bucăți:
# total_randuri / dimensiune_bucata

total_randuri = len(df_angajati)
dimensiune_bucata = 3
numar_bucati = math.ceil(total_randuri / dimensiune_bucata)
print(numar_bucati)
rezultate = []
reader = pd.read_csv('angajati.csv', chunksize=dimensiune_bucata)
import time

# Împachetăm iteratorul nostru în tqdm()
for bucata in tqdm(reader, total=numar_bucati, desc="Procesare angajați"):
    # Aici ar avea loc o procesare complexă
    # Pentru exemplu, doar adăugăm bucata la o listă
    rezultate.append(bucata)
    time.sleep(1)

df_final = pd.concat(rezultate)
print("\nDataFrame-ul a fost reconstruit din bucăți.")

# OBS.: Am folosit `total=numar_bucati` pentru a-i spune lui tqdm câte
# iterații vor fi în total. `desc` este un text descriptiv afișat în fața barei
# de progres.

2


Procesare angajați: 100%|██████████| 2/2 [00:02<00:00,  1.00s/it]


DataFrame-ul a fost reconstruit din bucăți.


In [17]:
# __EXERCIȚIU__
# Repetați exercițiul anterior (găsirea vechimii maxime folosind chunk
# processing), dar de data aceasta adăugați o bară de progres cu `tqdm` pentru
# a vizualiza procesul.

# HINT: Calculați `numar_bucati` și pasați iteratorul `pd.read_csv()` prin
# `tqdm()`, specificând `total` și un `desc`.


valoarea_maxima_globala = 0

dimensiune_bucata = 3
total_randuri = len(df_angajati)
numar_bucati = math.ceil(total_randuri / dimensiune_bucata)

reader = pd.read_csv('angajati.csv', chunksize=dimensiune_bucata)

import time

for bucata in tqdm(reader, total = numar_bucati, desc = "Calcul Vechime Maximă"):
  valoarea_maxima_locala = bucata["Vechime_ani"].max()
  #print(f"Valoarea maxima locala este: {valoarea_maxima_locala}")

  if valoarea_maxima_locala > valoarea_maxima_globala:
    valoarea_maxima_globala = valoarea_maxima_locala
    #print(f"Noul maxim a devenit: {valoarea_maxima_globala}")

  #print()
  time.sleep(1)

print()
print(f"Cea mai mare vechime este de {valoarea_maxima_globala} ani.")

Calcul Vechime Maximă..................: 100%|██████████| 2/2 [00:02<00:00,  1.00s/it]


Cea mai mare vechime este de 5 ani.


## Formate de Fișiere Eficiente: Parquet

Fișierele `.csv` sunt universale, dar nu sunt deloc eficiente. Ele stochează datele ca text, ocupă mult spațiu și sunt lente la citire. Pentru seturi de date mari, se folosesc formate de fișiere specializate.

**Parquet** este un format de fișier **columnar**, ceea ce înseamnă că stochează datele pe coloane, nu pe rânduri. Această abordare are avantaje uriașe:

* **Compresie mai bună**: Deoarece datele de pe o coloană sunt de același tip, pot fi comprimate mult mai eficient. Fișierele Parquet sunt adesea de câteva ori mai mici decât echivalentele lor CSV.
* **Citire mai rapidă**: Când realizăm o interogare, adesea avem nevoie doar de câteva coloane. Un format columnar permite citirea doar a acelor coloane, ignorând restul, ceea ce duce la o viteză de citire mult mai mare.
* **Păstrarea tipurilor de date**: Spre deosebire de CSV, Parquet salvează și tipurile de date, deci nu mai este nevoie să le specificăm la citire.

! Pentru a lucra cu Parquet, trebuie să instalați o bibliotecă precum `pyarrow`: `pip install pyarrow`.

In [18]:
# Exemplu: Salvarea și citirea unui fișier Parquet

# Să folosim DataFrame-ul nostru optimizat, df_optim
print("Tipuri de date înainte de salvare:")
print(df_optim.dtypes)

# Salvarea în format Parquet
df_optim.to_parquet('angajati.parquet', engine='pyarrow')

print("\nDataFrame-ul a fost salvat ca 'angajati.parquet'.")

# Citirea din formatul Parquet
df_citit_parquet = pd.read_parquet('angajati.parquet', engine='pyarrow')

print("\nTipuri de date după citirea din Parquet:")
print(df_citit_parquet.dtypes)

# OBS.: Observați cum tipurile de date optimizate ('category', 'float32',
# 'int8') au fost păstrate automat la citirea fișierului Parquet, un avantaj
# major față de CSV.

Tipuri de date înainte de salvare:
Nume                  object
Departament         category
Salariu_Brut_EUR     float32
Vechime_ani             int8
dtype: object

DataFrame-ul a fost salvat ca 'angajati.parquet'.

Tipuri de date după citirea din Parquet:
Nume                  object
Departament         category
Salariu_Brut_EUR     float32
Vechime_ani             int8
dtype: object


In [ ]:
# __EXERCIȚIU__
# 1. Creați un nou DataFrame numit `df_vanzari` cu următoarele date:
#    - 'Produs': ['Laptop', 'Mouse', 'Tastatura', 'Monitor', 'Laptop']
#    - 'Pret': [5500.50, 150.75, 250.00, 1200.50, 5700.00]
#    - 'Cantitate': [10, 50, 30, 15, 8]
# 2. Optimizați tipurile de date: 'Produs' -> 'category', 'Pret' -> 'float32',
# 'Cantitate' -> 'int16'.
# 3. Salvați DataFrame-ul optimizat într-un fișier numit 'vanzari.parquet'.
# 4. Citiți datele înapoi din 'vanzari.parquet' într-un nou DataFrame și
# afișați-l.

# HINT: Urmați pașii din exemplul anterior: creați DataFrame-ul, folosiți
# .astype() pentru fiecare coloană, apoi .to_parquet() și .read_parquet().

---
# Lucrul cu Arhive și Formate de Date Complexe

Pe măsură ce seturile de date devin mai mari, gestionarea fișierelor devine o provocare. Trimiterea unui fișier CSV de 2GB prin email este impracticabilă. Din acest motiv, datele sunt adesea arhivate pentru a reduce spațiul pe disc și pentru a facilita transferul. În acest capitol vom învăța cum să lucrăm direct cu fișiere arhivate și vom explora formate de date avansate, optimizate pentru performanță.

## Lucrul cu fișiere arhivate (.zip, .tar.gz)

Arhivarea fișierelor (cum ar fi în formate `.zip` sau `.gzip`) este o practică standard pentru a reduce dimensiunea datelor și pentru a grupa mai multe fișiere într-unul singur. Vestea bună este că Pandas poate citi direct din aceste arhive, fără a fi nevoie să le dezarhivăm manual.

Pentru formate precum `.zip` și `.gzip`, funcția `pd.read_csv()` poate gestiona direct calea către arhivă. Pentru arhive mai complexe precum `.tar`, am avea nevoie de o bibliotecă suplimentară precum `tarfile` pentru a extrage fișierul în memorie înainte de a-l citi cu Pandas.

In [21]:
# Exemplu 1: Citirea directă dintr-un fișier .zip

# Mai întâi, vom crea un fișier .zip pentru demonstrație.
df_produse.to_csv('produse.csv', index=False)

import zipfile
with zipfile.ZipFile('produse.zip', 'w') as zf:
    zf.write('produse.csv')

# Acum, citim direct din arhiva .zip
df_din_zip = pd.read_csv('produse.zip')

print("Datele citite din arhiva 'produse.zip':")
print(df_din_zip)

# OBS.: Pandas a decomprimat automat fișierul în memorie și l-a citit. Nu a
# fost nevoie de niciun pas manual de dezarhivare.

Datele citite din arhiva 'produse.zip':
      Produs     Categorie  Pret_RON  Stoc
0     Tricou  Îmbrăcăminte      80.0   100
1  Pantaloni  Îmbrăcăminte     250.0    50
2    Adidași  Încălțăminte     400.0    75
3      Șapcă     Accesorii      50.0   200
4      Geacă  Îmbrăcăminte     500.0    25
5    Hanorac  Îmbrăcăminte       NaN    40


In [ ]:
# __EXERCIȚIU__
# Scrieți codul necesar pentru a citi un fișier CSV numit 'date_clienti.csv'
# dintr-o arhivă numită 'clienti_arhiva.zip'.
# Stocați rezultatul într-un DataFrame numit `df_clienti`.

# HINT: Sintaxa este identică cu cea pentru un fișier CSV normal, doar calea
# se schimbă: `pd.read_csv('clienti_arhiva.zip')`.

## Formatul HDF5 (Hierarchical Data Format)

**HDF5** este un format de fișier și o bibliotecă concepute pentru a stoca și a organiza cantități mari de date. Spre deosebire de formatele simple precum CSV, HDF5 este un sistem de fișiere în sine, capabil să stocheze seturi de date multiple într-o structură ierarhică, similară cu folderele și fișierele de pe un computer. Este foarte eficient pentru seturi de date mari și complexe și a fost utilizat pe scară largă pentru stocarea modelelor în biblioteci de machine learning precum Keras.

! Pentru a lucra cu HDF5 în Pandas, trebuie instalată biblioteca `tables`: `pip install tables`.

In [22]:
# Exemplu 2: Salvarea și citirea unui fișier HDF5

# Salvarea DataFrame-ului într-un fișier HDF5
df_produse.to_hdf('magazin_date.h5', key='produse', format='table')

print("DataFrame-ul a fost salvat în 'magazin_date.h5' sub cheia 'produse'.")

# Citirea datelor din fișierul HDF5
df_din_hdf = pd.read_hdf('magazin_date.h5', 'produse')

print("\nDatele citite din HDF5:")
print(df_din_hdf)

# OBS.: 'key' funcționează ca un nume pentru setul de date în interiorul
# fișierului HDF5. Un singur fișier .h5 poate conține mai multe seturi de date,
# fiecare cu propria sa cheie.

DataFrame-ul a fost salvat în 'magazin_date.h5' sub cheia 'produse'.

Datele citite din HDF5:
      Produs     Categorie  Pret_RON  Stoc
0     Tricou  Îmbrăcăminte      80.0   100
1  Pantaloni  Îmbrăcăminte     250.0    50
2    Adidași  Încălțăminte     400.0    75
3      Șapcă     Accesorii      50.0   200
4      Geacă  Îmbrăcăminte     500.0    25
5    Hanorac  Îmbrăcăminte       NaN    40


In [ ]:
# __EXERCIȚIU__
# Salvați DataFrame-ul `df_angajati` într-un fișier HDF5 numit 'companie.h5',
# folosind cheia 'departament_it'.
# Apoi, citiți datele înapoi într-un nou DataFrame și afișați-l pentru a
# verifica.

# HINT: Folosiți `df_angajati.to_hdf('companie.h5', key='departament_it')` și
# apoi `pd.read_hdf('companie.h5', 'departament_it')`.

## Apache Arrow

**Apache Arrow** nu este un format de fișier, ci o platformă de dezvoltare pentru lucrul cu date **în memorie**. Scopul său principal este de a standardiza formatul datelor tabulare în memoria RAM, astfel încât diferite sisteme (Pandas, Spark, baze de date) să poată schimba date între ele fără costisitoarele procese de serializare și deserializare (conversia datelor într-un format care poate fi stocat sau transmis și apoi înapoi).

Pandas se integrează din ce în ce mai strâns cu PyArrow (implementarea Python pentru Arrow) pentru a oferi performanțe îmbunătățite și tipuri de date mai avansate, care pot gestiona mai bine valorile lipsă și pot accelera operațiunile.

! Pentru a folosi Arrow, trebuie instalată biblioteca `pyarrow`: `pip install pyarrow`.

In [ ]:
# Exemplu: Conversia între Pandas DataFrame și Arrow Table
import pyarrow as pa

# 1. Convertim un DataFrame Pandas într-un tabel Arrow
tabel_arrow = pa.Table.from_pandas(df_produse)

print("DataFrame-ul a fost convertit într-un tabel Arrow.")
print("Schema tabelului Arrow:")
print(tabel_arrow.schema)

# 2. Convertim tabelul Arrow înapoi într-un DataFrame Pandas
df_inapoi_in_pandas = tabel_arrow.to_pandas()

print("\nTabelul Arrow a fost convertit înapoi în Pandas:")
print(df_inapoi_in_pandas)

# OBS.: Conversia este foarte rapidă deoarece ambele biblioteci pot partaja același format de date în memorie.
# Acest lucru este crucial pentru interoperabilitatea în ecosistemele de big data. [cite: 292]

DataFrame-ul a fost convertit într-un tabel Arrow.
Schema tabelului Arrow:
Produs: string
Categorie: string
Pret_RON: double
Stoc: int64
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 711

Tabelul Arrow a fost convertit înapoi în Pandas:
      Produs     Categorie  Pret_RON  Stoc
0     Tricou  Îmbrăcăminte      80.0   100
1  Pantaloni  Îmbrăcăminte     250.0    50
2    Adidași  Încălțăminte     400.0    75
3      Șapcă     Accesorii      50.0   200
4      Geacă  Îmbrăcăminte     500.0    25
5    Hanorac  Îmbrăcăminte       NaN    40


In [ ]:
# Exemplu: Crearea unei coloane cu tip de date PyArrow-backed

serie_arrow = pd.Series([-2.5, 10.1, None], dtype="float32[pyarrow]")

print("O serie Pandas cu tip de date susținut de PyArrow:")
print(serie_arrow)

# OBS.: Observați cum valoarea lipsă este reprezentată ca `<NA>`, specific
# tipurilor de date Arrow, ceea ce oferă o gestionare mai consistentă a datelor
# lipsă.

O serie Pandas cu tip de date susținut de PyArrow:
0    -2.5
1    10.1
2    <NA>
dtype: float[pyarrow]


In [ ]:
# __EXERCIȚIU__
# 1. Convertiți DataFrame-ul `df_angajati` într-un tabel Apache Arrow.
# 2. Afișați schema tabelului Arrow pentru a observa tipurile de date.
# 3. Convertiți tabelul înapoi într-un DataFrame Pandas și afișați-l.

# HINT: Folosiți `pa.Table.from_pandas()` și `tabel.to_pandas()`.